In [1]:
%%capture
%run utils.ipynb
%run data.ipynb

import warnings
warnings.filterwarnings("ignore")

# Delta-vega hedging

$$\Delta=\frac{\partial C^{BS}}{}$$

In [2]:
isx15012010C = get_sheet('isx15012010C')
isx19022010C = get_sheet('isx19022010C')

isx_m45 = isx15012010C[isx15012010C.shape[0]-45:].reset_index()
sp500_spot = isx_m45.iloc[0]['sp500_spot'] # S&P500 spot

In [3]:
isx_E520_C = isx15012010C[['days_to_mat',520,'sp500_spot','rf','date']].loc[20:]
isx_E520_C.head()

,days_to_mat,520,sp500_spot,rf,date
20,66,11.45,506.75,0.07,19.10.2009
21,65,10.80,504.52,0.07,20.10.2009
22,64,9.30,500.24,0.07,21.10.2009
23,63,10.70,504.84,0.07,22.10.2009
24,62,9.10,500.00,0.07,23.10.2009


In [4]:
isx19022010C.rename(columns={'Unnamed: 56': 'date'}, inplace=True)
isx_E520_R = isx19022010C[['days_to_mat',520,'sp500_spot','rf','date']]
isx_E520_R.head()

,days_to_mat,520,sp500_spot,rf,date
0,90,15.05,506.75,0.07,19.10.2009
1,89,14.90,504.52,0.07,20.10.2009
2,88,12.70,500.24,0.07,21.10.2009
3,87,14.40,504.84,0.07,22.10.2009
4,86,12.70,500.00,0.07,23.10.2009


In [5]:
# Rowwise computations of implied vola and delta
def implied_vola_rows(E,row):  return solve_implied_vola(row[E],row['sp500_spot'],E,row['days_to_mat'],row['rf'])
def delta_rows(E,row): return delta_call(row['sp500_spot'],E,row['days_to_mat'],row['rf'],row['implied_vola='+str(E)])
def vega_rows(E,row): return vega_call(row['sp500_spot'],E,row['days_to_mat'],row['rf'],row['implied_vola='+str(E)])

## Rehedging frequency: 1 day

In [6]:
isx_E520_C['implied_vola=520'] = isx_E520_C.apply(lambda row: implied_vola_rows(520,row), axis=1)
isx_E520_C['delta=520'] = isx_E520_C.apply(lambda row: delta_rows(520,row), axis=1)
isx_E520_C['vega=520'] = isx_E520_C.apply(lambda row: vega_rows(520,row), axis=1)
isx_E520_C.shape
isx_E520_C.tail()

,days_to_mat,520,sp500_spot,rf,date,implied_vola=520,delta=520,vega=520
81,5,6.8,524.29,0.06,12.1.2010,1.644438e-01,0.684107,21.824942
82,4,9.0,527.93,0.06,13.1.2010,1.537721e-01,0.838691,13.518695
83,3,10.1,529.59,0.06,14.1.2010,1.472639e-01,0.921081,7.064685
84,2,4.1,524.11,0.05,15.1.2010,3.781978e+12,1.000000,0.000000
85,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
isx_E520_R['implied_vola=520'] = isx_E520_R.apply(lambda row: implied_vola_rows(520,row), axis=1)
isx_E520_R['delta=520'] = isx_E520_R.apply(lambda row: delta_rows(520,row), axis=1)
isx_E520_R['vega=520'] = isx_E520_R.apply(lambda row: vega_rows(520,row), axis=1)
isx_E520_R.rename(columns={'vega=520': 'R_vega=520', 'delta=520': 'R_delta=520', 520 : 'R_520', 'sp500_spot': 'R_sp500_spot'}, inplace=True)

In [8]:
isx_E520_R.shape
isx_E520_R = isx_E520_R.iloc[:65]

In [9]:
hedge = pd.concat([isx_E520_C.reset_index(),isx_E520_R.reset_index()], axis=1)[[520,'R_520','sp500_spot','delta=520','vega=520','R_delta=520','R_vega=520']]

In [10]:
hedge['alpha'] = hedge.apply(lambda row: row['delta=520']+row['R_delta=520']*row['vega=520']/row['R_vega=520'], axis=1)

In [11]:
hedge['eta'] = hedge.apply(lambda row: row['vega=520']/row['R_vega=520'], axis=1)

In [12]:
hedge['hedge_pos'] = hedge.apply(lambda row: row['alpha']*row['sp500_spot']+row['eta']*row['R_520'], axis=1)

In [13]:
hedge['change_P'] = hedge[520].diff()
hedge['change_R'] = hedge['hedge_pos'].diff()
hedge['MSE'] = hedge.apply(lambda row: (row['change_P']-row['change_R'])**2,axis=1)
hedge['MSE'].sum()/hedge['MSE'].count()

2311.2304967831205

## Rehedging frequency: 3 days

In [24]:
hedge['delta_freq'] = set_rehedge_freq(3,hedge['delta=520'])
hedge['vega_freq'] = set_rehedge_freq(3,hedge['vega=520'])
hedge['R_delta_freq'] = set_rehedge_freq(3,hedge['R_delta=520'])
hedge['R_vega_freq'] = set_rehedge_freq(3,hedge['R_vega=520'])

In [25]:
hedge['alpha'] = hedge.apply(lambda row: row['delta_freq']+row['R_delta_freq']*row['vega_freq']/row['R_vega_freq'], axis=1)
hedge['eta'] = hedge.apply(lambda row: row['vega_freq']/row['R_vega_freq'], axis=1)
hedge['hedge_pos'] = hedge.apply(lambda row: row['alpha']*row['sp500_spot']+row['eta']*row['R_520'], axis=1)

In [26]:
hedge['change_P'] = hedge[520].diff()
hedge['change_R'] = hedge['hedge_pos'].diff()
hedge['MSE'] = hedge.apply(lambda row: (row['change_P']-row['change_R'])**2,axis=1)
hedge['MSE'].sum()/hedge['MSE'].count()

2057.0960434274157

## Rehedging frequency: 5 days

In [17]:
hedge['delta_freq'] = set_rehedge_freq(5,hedge['delta=520'])
hedge['vega_freq'] = set_rehedge_freq(5,hedge['vega=520'])
hedge['R_delta_freq'] = set_rehedge_freq(5,hedge['R_delta=520'])
hedge['R_vega_freq'] = set_rehedge_freq(5,hedge['R_vega=520'])

In [18]:
hedge['alpha'] = hedge.apply(lambda row: row['delta_freq']+row['R_delta_freq']*row['vega_freq']/row['R_vega_freq'], axis=1)
hedge['eta'] = hedge.apply(lambda row: row['vega_freq']/row['R_vega_freq'], axis=1)
hedge['hedge_pos'] = hedge.apply(lambda row: row['alpha']*row['sp500_spot']+row['eta']*row['R_520'], axis=1)

In [19]:
hedge['change_P'] = hedge[520].diff()
hedge['change_R'] = hedge['hedge_pos'].diff()
hedge['MSE'] = hedge.apply(lambda row: (row['change_P']-row['change_R'])**2,axis=1)
hedge['MSE'].sum()/hedge['MSE'].count()

1078.3941141998473

## In the money results

## Rehedging frequency: 1 days

In [14]:
isx_E530_C = isx15012010C[['days_to_mat',530,'sp500_spot','rf','date']].loc[20:]
isx_E530_R = isx19022010C[['days_to_mat',530,'sp500_spot','rf','date']]

In [15]:
isx_E530_C['implied_vola=530'] = isx_E530_C.apply(lambda row: implied_vola_rows(530,row), axis=1)
isx_E530_C['delta=530'] = isx_E530_C.apply(lambda row: delta_rows(530,row), axis=1)
isx_E530_C['vega=530'] = isx_E530_C.apply(lambda row: vega_rows(530,row), axis=1)
isx_E530_C.shape
isx_E530_C.tail()

,days_to_mat,530,sp500_spot,rf,date,implied_vola=530,delta=530,vega=530
81,5,1.00,524.29,0.06,12.1.2010,1.181703e-01,0.236736,18.933754
82,4,1.60,527.93,0.06,13.1.2010,1.069394e-01,0.387735,21.169175
83,3,1.95,529.59,0.06,14.1.2010,1.056270e-01,0.490216,19.148440
84,2,0.05,524.11,0.05,15.1.2010,2.846228e+12,1.000000,0.000000
85,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
isx_E530_R['implied_vola=530'] = isx_E530_R.apply(lambda row: implied_vola_rows(530,row), axis=1)
isx_E530_R['delta=530'] = isx_E530_R.apply(lambda row: delta_rows(530,row), axis=1)
isx_E530_R['vega=530'] = isx_E530_R.apply(lambda row: vega_rows(530,row), axis=1)
isx_E530_R.rename(columns={'vega=530': 'R_vega=530', 'delta=530': 'R_delta=530', 530 : 'R_530', 'sp500_spot': 'R_sp500_spot'}, inplace=True)

In [17]:
isx_E530_R.shape
isx_E530_R = isx_E530_R.iloc[:65]

In [18]:
hedge2 = pd.concat([isx_E530_C.reset_index(),isx_E530_R.reset_index()], axis=1)[[530,'R_530','sp500_spot','delta=530','vega=530','R_delta=530','R_vega=530']]

In [19]:
hedge2['alpha'] = hedge2.apply(lambda row: row['delta=530']+row['R_delta=530']*row['vega=530']/row['R_vega=530'], axis=1)
hedge2['eta'] = hedge2.apply(lambda row: row['vega=530']/row['R_vega=530'], axis=1)
hedge2['hedge_pos'] = hedge2.apply(lambda row: row['alpha']*row['sp500_spot']+row['eta']*row['R_530'], axis=1)

In [20]:
hedge2['change_P'] = hedge2[530].diff()
hedge2['change_R'] = hedge2['hedge_pos'].diff()
hedge2['MSE'] = hedge2.apply(lambda row: (row['change_P']-row['change_R'])**2,axis=1)
hedge2['MSE'].sum()/hedge['MSE'].count()

17628.58389814175

## Rehedging frequency: 3 days

In [33]:
hedge2['delta_freq'] = set_rehedge_freq(3,hedge2['delta=530'])
hedge2['vega_freq'] = set_rehedge_freq(3,hedge2['vega=530'])
hedge2['R_delta_freq'] = set_rehedge_freq(3,hedge2['R_delta=530'])
hedge2['R_vega_freq'] = set_rehedge_freq(3,hedge2['R_vega=530'])

In [34]:
hedge2['alpha'] = hedge2.apply(lambda row: row['delta_freq']+row['R_delta_freq']*row['vega_freq']/row['R_vega_freq'], axis=1)
hedge2['eta'] = hedge2.apply(lambda row: row['vega_freq']/row['R_vega_freq'], axis=1)
hedge2['hedge_pos'] = hedge2.apply(lambda row: row['alpha']*row['sp500_spot']+row['eta']*row['R_530'], axis=1)

In [35]:
hedge2['change_P'] = hedge2[530].diff()
hedge2['change_R'] = hedge2['hedge_pos'].diff()
hedge2['MSE'] = hedge2.apply(lambda row: (row['change_P']-row['change_R'])**2,axis=1)
hedge2['MSE'].sum()/hedge2['MSE'].count()

8623.593176065791

## Rehedging frequency: 5 days

In [21]:
hedge2['delta_freq'] = set_rehedge_freq(5,hedge2['delta=530'])
hedge2['vega_freq'] = set_rehedge_freq(5,hedge2['vega=530'])
hedge2['R_delta_freq'] = set_rehedge_freq(5,hedge2['R_delta=530'])
hedge2['R_vega_freq'] = set_rehedge_freq(5,hedge2['R_vega=530'])

In [22]:
hedge2['alpha'] = hedge2.apply(lambda row: row['delta_freq']+row['R_delta_freq']*row['vega_freq']/row['R_vega_freq'], axis=1)
hedge2['eta'] = hedge2.apply(lambda row: row['vega_freq']/row['R_vega_freq'], axis=1)
hedge2['hedge_pos'] = hedge2.apply(lambda row: row['alpha']*row['sp500_spot']+row['eta']*row['R_530'], axis=1)

In [23]:
hedge2['change_P'] = hedge2[530].diff()
hedge2['change_R'] = hedge2['hedge_pos'].diff()
hedge2['MSE'] = hedge2.apply(lambda row: (row['change_P']-row['change_R'])**2,axis=1)
hedge2['MSE'].sum()/hedge2['MSE'].count()

3561.6257415447767